In [ ]:
#Imports

from __future__ import print_function

from random import random
from datetime import datetime
import sys
import re
import webbrowser
import time

import os
import pickle
import google.oauth2.credentials
from email.mime.text import MIMEText
import base64

from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials

In [ ]:
#register chrome webbrowser
webbrowser.register('chrome', None, webbrowser.BackgroundBrowser("C://Program Files (x86)//Google//Chrome//Application//chrome.exe"))

In [ ]:
#we can read and write messages to our GMail
SCOPES = [
    'https://www.googleapis.com/auth/gmail.readonly'
]

In [ ]:
def get_authenticated_gmail_service():
    """
    This function uses the token.json file to get an authenticated object used to talk to Gmail
    """

    creds = None
    API_SERVICE_NAME = 'gmail'
    API_VERSION = 'v1'
    
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', 
                SCOPES
            )
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    return build('gmail', 'v1', credentials=creds)

In [ ]:
#get the service
service = get_authenticated_gmail_service()  

In [ ]:
#get me last 3mo of messages which are in promotions folder and 
#which contain "unsubscribe" as an option
result = service.users().messages().list(
    userId='me', 
    q='"unsubscribe" AND category:promotions AND newer_than:90d'
).execute()

In [ ]:
#number of links we've opened
curr_open = 0

#avoid opening duplicate links
seen_urls = set()

#for each message...
for message in result['messages']:
    #get the id
    message_id = message.get('id')
    
    #get all info about that message
    message_info =  service.users().messages().get(userId='me', id=message_id).execute()
    
    #for each header...
    for h in message_info.get('payload').get('headers'):
        #look out of the List-Unsubscribe header
        if h.get('name') == 'List-Unsubscribe':
            #get this url
            url = re.findall(r'\<(.*?)\>', h.get('value'))[0]

            #find first url with any of these extensions
            for ext in ['.com', '.edu', '.net', '.gov']:
                base_url = re.findall(f'(.*?{ext})', url)
                if len(base_url) > 0:
                    base = base_url[0]
                    break
            
            #open the url in chrome
            if url[:4] == 'http' and base not in seen_urls:
                webbrowser.get('chrome').open(url)
                seen_urls.add(base)
                curr_open += 1

    #can open max of 3 at a time so we don't spam the browser
    if curr_open == 3:
        answer = input('opened 3 tabs. type "yes" to continue. type "no" to stop. ')
        if answer == "yes":
            curr_open = 0
        else:
            break